In [ ]:

# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json
from requests import get
from time import sleep
from pprint import pprint
from config import api_key
from datetime import datetime


In [ ]:

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)

lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)


In [ ]:

# Setup the cities iterator
city_iter = [str(i) for i in cities]

# Initialize needed objects
city_count = 0
city_data = pd.DataFrame()


In [ ]:

# For loop through the cities iterator, pull needed data from the JSON request object
for city in city_iter:
    
    response = get('http://api.openweathermap.org/data/2.5/weather?q='
                   + city
                   + '&APPID='
                   + api_key).json()
    
    # Pause for 1.175 seconds (70.5 seconds for 60 requests) 
    # to ensure compliance with request rate of 60 calls per minute
    sleep(1.175)
    
    try:
        
        # Pull needed data from JSON response object and insert into a dictionary
        city_dict = {
            'city_id': response['id'],
            'name': response['name'],
            'country': response['sys']['country'],
            'longitude': response['coord']['lon'],
            'latitude': response['coord']['lat'],
            'cloudiness': response['clouds']['all'],
            'humidity': response['main']['humidity'],
            'temperature': response['main']['temp'],
            'wind_speed': response['wind']['speed'],
            'time': response['dt']
        }
        
        # Create a temporary single row dataframe with the current cities data
        temp_df = pd.DataFrame(city_dict, index = ['city']).set_index('city_id')  
        
        # Add the temporary dataframe row to the pre-intialized city_data DataFrame
        city_data = pd.concat([city_data, temp_df])
        
        # Add 1 for each city iteration to the city counter
        city_count += 1
        
        # Collect the current cities name
        city_name = city_dict['name']    
            
        # Re-intialise the city dictionary to be empty for the next interation
        city_dict = {}
    
    # Raise a KeyError if a cities JSON response does not include any of the needed data
    except KeyError as e:
        
        print(f'Skipping city ... : KeyError: {e} not found.') 
    
    print(f'Accessing city # {city_count} | {city_name}')


#### Post API call inspection

In [ ]:
len(city_data)

In [ ]:
city_data.info()

In [ ]:
city_data.sample(20)

#### Write to CSV

In [ ]:
city_data.to_csv('city_data.csv')